<a href="https://colab.research.google.com/github/Vinayak-Sharma12/GENAI-II/blob/main/5_YOUTUBE_VIDEO_CHATBOT(RAG).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q langchain langchain_community langchain_groq  faiss-cpu youtube-transcript-api

### **YOUTUBE VIDEO TRANSCRIPT**

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi,TranscriptsDisabled

In [ ]:
url='https://www.youtube.com/watch?v=Pk2BeaGbcTE'
splits=url.split('v=')

In [ ]:
splits

In [ ]:
splits[1]

In [ ]:
video_id = "Gfr50f6ZBvo" # only the ID, not full URL
try:
    # If you don’t care which language, this returns the “best” one
    transcript_list = YouTubeTranscriptApi.get_transcript(video_id, languages=["en"])

    # Flatten it to plain text
    transcript = " ".join(chunk["text"] for chunk in transcript_list)
    print(transcript)

except TranscriptsDisabled:
    print("No captions available for this video.")

In [ ]:
transcript_list

In [ ]:
transcript

In [ ]:
len(transcript)

### **TEXT SPLITTING**

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
chunks = splitter.create_documents([transcript])

In [ ]:
len(chunks)

### **VECTOR STORE(FAISS)**

FAISS WORKING(Important):

        So we have chunks as document object which have page_content and
        metadata and now when we use FAISS vector store what it do is it stores
        the embeddings of each chunk and gives each chunk unique id to that
        chunk but don't append that it in chunk but when we try to retrieve
        from FAISS vector storee it returns a document object with page_content,
        meta data (by getting it from chunks): and unique id

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')

In [ ]:
vector_store=FAISS.from_documents(chunks,embeddings)

In [ ]:
vector_store.index_to_docstore_id

In [ ]:
len(vector_store.index_to_docstore_id)

In [ ]:
vector_store.get_by_ids(['e6f3e5af-b1cc-42b4-833c-1a029133e9f5'])

In [ ]:
chunks[0]

### **RETRIEVER**

RERIEVER first takes the user query convert it into the embeddings using same embedding model as used by vector store and then perform semantic search in Vector Store

In [ ]:
retriever=vector_store.as_retriever(search_kwargs={'k':4})

To extract text from each doc received from retriever

In [ ]:
def format_docs(retrieved_docs):
  context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
  return context_text

### **PROMPT**

In [ ]:
from langchain.prompts import PromptTemplate
prompts=PromptTemplate(
      template="""You are helpful assisstant
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.
      {context}
      Question:{question}
      """,
      input_variables=['context','question']
)

### **CHAIN PART**

In [ ]:
from google.colab import userdata
key=userdata.get('groq_api_key')

In [ ]:
from langchain_groq import ChatGroq
llm=ChatGroq(model='llama-3.1-8b-instant',api_key=key)

In [ ]:
from langchain_core.runnables import RunnableParallel,RunnablePassthrough,RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [ ]:
parser=StrOutputParser()

In [ ]:
parallel_chain=RunnableParallel(
    {
    'question':RunnablePassthrough(),
     'context':retriever|RunnableLambda(format_docs)
    }
)

In [ ]:
chain=parallel_chain|prompts|llm|parser

In [ ]:
chain.invoke('Can you summarize the video')